In [1]:
# bring in dependencies
import pandas as pd
import numpy as np
import json
import pprint

In [2]:
year_list = ["2015", "2016", "2017", "2018", "2019", "2020", "2021"]
happiness_frame = []
happiness_df = pd.DataFrame()

file_path = "../happiness_data/"

for y in year_list:
    csv_file = file_path + y + ".csv"
#     year = y
#     print(csv_file)
    year = pd.read_csv(csv_file)
    year["yr"] = y
    
    year = year.transpose()
    year.reset_index(inplace=True)
    year.rename(columns = {"index": "col_name"}, inplace=True)
    year = year[year["col_name"].str.contains("Region|Rank|RANK|Explain|Standard|Whisker|Confidence|Dystopia|Overall|whisker") == False]
    year["new_col_name"] = np.where(year.col_name.str.contains("GDP"), "GDP per Capita",
        np.where(year.col_name.str.contains("Happiness"),"Happiness", 
        np.where(year.col_name.str.contains("Ladder"),"Happiness",
        np.where(year.col_name.str.contains("life expectancy"),"Life Expectancy",
        np.where(year.col_name.str.contains("Freedom"),"Freedom",
        np.where(year.col_name.str.contains("corruption"),"Corruption",
        np.where(year.col_name.str.contains("score"),"Happiness",
        np.where(year.col_name.str.contains("Family"),"Social Support",
        np.where(year.col_name.str.contains("yr"),"Year",
        np.where(year.col_name.str.contains("Country"),"Country",
        np.where(year.col_name.str.contains("Region"),"Region",
        np.where(year.col_name.str.contains("Corruption"),"Corruption",
        np.where(year.col_name.str.contains("Expectancy"),"Life Expectancy",
        np.where(year.col_name.str.contains("Social"),"Social Support",
        np.where(year.col_name.str.contains("Generosity"),"Generosity",
        np.where(year.col_name.str.contains("Score"),"Happiness",
        "Other"))))))))))))))))
    year.drop(columns=['col_name'], inplace=True)
    year = year.transpose()
    year.set_axis(["Country", "Happiness", "GDP per Capita", "Social Support", "Life Expectancy", "Freedom", "Corruption", "Generosity", "Year"], axis=1,inplace = True)
    year = year[:-1]
    year_df = pd.DataFrame(year)
    happiness_df = pd.concat([happiness_df, year_df])  
    
happiness_df.to_json(r'../happiness_data/happiness_json.json', orient='records')

In [3]:
file = "../happiness_csvs/countries_latitude_longitude.csv"
latlng = pd.read_csv(file)
latlng.rename(columns = {"country_name": "Country"}, inplace=True)
latlng

,Country,latitude,longitude
0,Afghanistan,33.939110,67.709953
1,Albania,41.153332,20.168331
2,Algeria,28.033886,1.659626
3,Angola,-11.202692,17.873887
4,Argentina,-38.416097,-63.616672
...,...,...,...
160,Venezuela,6.423750,-66.589730
161,Vietnam,14.058324,108.277199
162,Yemen,15.552727,48.516388
163,Zambia,-13.133897,27.849332


In [20]:
merged_happiness = pd.merge(happiness_df, latlng, how = "outer", on = "Country")
merged_happiness

,Country,Happiness,GDP per Capita,Social Support,Life Expectancy,Freedom,Corruption,Generosity,Year,latitude,longitude
0,Switzerland,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2015,46.818188,8.227512
1,Switzerland,7.509,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2016,46.818188,8.227512
2,Switzerland,7.494,1.56498,1.516912,0.858131,0.620071,0.290549,0.367007,2017,46.818188,8.227512
3,Switzerland,7.487,1.42,1.549,0.927,0.66,0.256,0.357,2018,46.818188,8.227512
4,Switzerland,7.48,1.452,1.526,1.052,0.572,0.263,0.343,2019,46.818188,8.227512
...,...,...,...,...,...,...,...,...,...,...,...
1079,Gambia,5.051,7.686,0.69,55.16,0.697,0.424,0.746,2021,13.443182,-15.310139
1080,Hong Kong S.A.R. of China,5.5104,10.934671,0.845969,76.771706,0.779834,0.13498,0.420607,2020,NaN,NaN
1081,Hong Kong S.A.R. of China,5.477,11.0,0.836,76.82,0.717,0.067,0.403,2021,NaN,NaN
1082,Maldives,5.1976,9.518633,0.913161,70.599998,0.853963,0.038085,0.824211,2020,3.202778,73.220680


In [21]:
cols = merged_happiness.columns.difference(["Country", "latitude", "longitude"])
j = (merged_happiness.groupby(["Country", "latitude", "longitude"])[cols]
       .apply(lambda x: x.to_dict('r'))
       .reset_index(name= "Metrics")
       .to_json(r'../happiness_data/j.json',orient='records'))

<ipython-input-21-39f932b7907e>:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  .apply(lambda x: x.to_dict('r'))
